# Because Cambridge is not "an African even-toed ungulate mammal"

This is a word perturbation analysis, originally reported in the [DESM](https://arxiv.org/abs/1602.01137v1.pdf) (2016) paper. The demo shows how a distributed representation (or embedding) based model collects evidence on the relevance of a document to a query by investigating all words in the document.

## Let's start
We begin by including the necessary modules and defining some utility functions.

In [1]:
import csv
import re
import math
from IPython.display import HTML, display

def cosine_sim(a, b):
    if a == None or b == None:
        return 0
    
    d = dot_product(a, b);
    na = max(1e-20, norm(a));
    nb = max(1e-20, norm(b));
    return (d / (na * nb) + 1) / 2;

def dot_product(a, b):
    result = 0;
    for i in range(len(a)):
        result += (a[i] * b[i])
    return result;
    
def norm(a):
    return math.sqrt(dot_product(a, a))
    
def centroid(a):
    dim = len(a[0])
    num = len(a)
    b = []
    for i in range(dim):
        b.append(0)
        for j in range(num):
            b[i] += (a[j][i] / num)
    return b

def get_clean(s):
    return re.sub('[^0-9a-z\t]+', '', s.lower())
    
def get_pre_punc(s):
    c = s[0].lower()
    o = ord(c)
    if (o < ord('a') or o > ord('z')) and (o < ord('0') or o > ord('9')):
        return c
    else:
        return ""

def get_post_punc(s):
    c = s[len(s)-1].lower()
    o = ord(c)
    if (o < ord('a') or o > ord('z')) and (o < ord('0') or o > ord('9')):
        return c
    else:
        return ""

## The data

Let's consider the query "cambridge" and the following three passages to be ranked for the query.

In [2]:
query = "cambridge"

passage = []
passage.append("The city of Cambridge is a university city and the county town of Cambridgeshire, England. It lies in East Anglia, on the River Cam, about 50 miles (80 km) north of London. According to the United Kingdom Census 2011, its population was 123867 (including 24488 students). This makes Cambridge the second largest city in Cambridgeshire after Peterborough, and the 54th largest in the United Kingdom. There is archaeological evidence of settlement in the area during the Bronze Age and Roman times; under Viking rule Cambridge became an important trading centre. The first town charters were granted in the 12th century, although city status was not conferred until 1951.")
passage.append("Oxford is a city in the South East region of England and the county town of Oxfordshire. With a population of 159994 it is the 52nd largest city in the United Kingdom, and one of the fastest growing and most ethnically diverse. Oxford has a broad economic base. Its industries include motor manufacturing, education, publishing and a large number of information technology and science-based businesses, some being academic offshoots. The city is known worldwide as the home of the University of Oxford, the oldest university in the English-speaking world. Buildings in Oxford demonstrate examples of every English architectural period since the arrival of the Saxons, including the mid-18th-century Radcliffe Camera. Oxford is known as the city of dreaming spires, a term coined by poet Matthew Arnold.")
passage.append("The giraffe (Giraffa camelopardalis) is an African even-toed ungulate mammal, the tallest living terrestrial animal and the largest ruminant. Its species name refers to its camel-like shape and its leopard-like colouring. Its chief distinguishing characteristics are its extremely long neck and legs, its horn-like ossicones, and its distinctive coat patterns. It is classified under the family Giraffidae, along with its closest extant relative, the okapi. The nine subspecies are distinguished by their coat patterns. The scattered range of giraffes extends from Chad in the north to South Africa in the south, and from Niger in the west to Somalia in the east. Giraffes usually inhabit savannas, grasslands, and open woodlands.")

passage_name = []
passage_name.append("Passage about the city of Cambridge")
passage_name.append("Passage about the city of Oxford")
passage_name.append("Passage about giraffes")

Additionally, let's construct two more passages by replacing the word 'giraffe' with 'cambridge' in the passage about giraffes, and by replacing the word 'cambridge' with 'giraffe' in the passage about Cambridge.

In [3]:
passage.append(re.sub(r"\b[Gg]iraffe\b", "Cambridge", passage[2]))
passage.append(re.sub(r"\bCambridge\b", "giraffe", passage[0]))

passage_name.append("Passage about giraffes, but 'giraffe' is replayed by 'Cambridge'")
passage_name.append("Passage about the city of Cambridge, but 'Cambridge' is replaced by 'giraffe'")

Now let's make a vocabulary list of all words from our query and passages.

In [4]:
doc_all = " ".join(passage)
words = re.sub('[^0-9a-z\t]+', ' ', doc_all.lower()).split()
seen = set()
vocab = [x for x in words if not (x in seen or seen.add(x))]

Last but not the least, let's load all the IN and OUT embedding vectors for our vocabulary words.

**Download** the DESM word embeddings data used in this demo from [here](https://www.microsoft.com/en-us/download/details.aspx?id=52597) for this step.

In [5]:
in_vec = {}
out_vec = {}

with open("data\\in.txt", mode='r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if row[0] in vocab:
            in_vec[row[0]] = [float(row[i]) for i in range(1, 201)]

with open("data\\out.txt", mode='r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if row[0] in vocab:
            out_vec[row[0]] = [float(row[i]) for i in range(1, 201)]

# Time to have some fun

Now that we have loaded the word embeddings in memory, we compute the centroid of the IN vectors of the query words and compare the OUT vectors corresponding to every word in the passages to the query centroid. The word cloud based on the cosine similarities is shown below. Note that we apply an exponential function over the scores to accentuate the difference for rendering the word cloud.

In [6]:
def get_font(x):
    f = 13
    x = math.exp(f*(x + 1) / 2) / math.exp(f)
    size = x * 4000
    return "style=\"font-size:{}%;color:#444444\"".format(size)

q_vec = centroid([in_vec[word] for word in query.split()])
display(HTML("<br /><h3>Query: \"cambridge\"</h3>" + "".join("<br /><h4>{}</h4><p style=\"border-left: 6px solid blue; background-color: #fafafa\"><br /><br />".format(passage_name[i]) + " ".join("{}<span {}>{}</span>{}".format(get_pre_punc(word), get_font(cosine_sim(q_vec, out_vec.get(get_clean(word), None))), get_clean(word), get_post_punc(word)) for word in passage[i].split()) + "<br /><br /></p>" for i in range(5))))


This examples highlights how by inspecting all the words in the document the DESM models can become more robust towards irrelevant terms. For example, when the word "giraffe" is replaced by the word "cambridge", the passage on giraffes is still scored low by the DESM for the query "cambridge" because it finds low supporting evidence from the other words in the passage. However, the DESM confuses the passage about Oxford to be relevant for the query "cambridge" because it detects a high number of similar words in the passage that frequently co-occur with the word "Cambridge". Note that this holds true for both IN-OUT and IN-IN cosine similarities.